In [ ]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import cv2 as cv
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from IPython.display import HTML
import matplotlib.patches as patches
%matplotlib widget

In [ ]:
img = cv.imread('/misc/home/u1220/analysis_jupyter/_trash/scar/scar_rupa-1.jpg')
imgray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(imgray, 127, 255, 0)
im2, contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

In [ ]:
cnt = contours[3]
cntrs = cv.drawContours(img, [cnt], 0, (255,0,0), 3)
fig = figure(num=None, figsize=(6, 6), dpi=80, facecolor='w')
plt.imshow(cv.cvtColor(imgray, cv.COLOR_GRAY2RGB))

In [ ]:
from shapely import affinity
from shapely.geometry import MultiPoint, Polygon, Point

my_cnt = contours[3]
np_cnt= np.array([c[0] for c in my_cnt])
fig = figure(num=None, figsize=(10, 6), dpi=80, facecolor='w')

# right wing
np_cnt = np_cnt[(np_cnt[:,0] <= 104) & (np_cnt[:,1] >= 47)]

# scaling down
len_x, len_y = 2*(np.max(np_cnt[:,0])-np.min(np_cnt[:,0])), 2*(np.max(np_cnt[:,1])-np.min(np_cnt[:,1]))
real_x = 40
real_y = real_x*(len_y/len_x)
scale_x, scale_y = real_x/len_x, real_y/len_y
np_cnt = np.array(affinity.scale(MultiPoint(np_cnt), xfact=scale_x, yfact=scale_y, origin=(np.min(np_cnt[:,0]),np.min(np_cnt[:,1]))))

# right top
ref_cnt = np.array(affinity.scale(MultiPoint(np_cnt), xfact=-1, yfact=1)) # reflecting right top wing
ref_cnt[:,0] = ref_cnt[:,0]+(np.max(np_cnt[:,0])-np.min(np_cnt[:,0]))
ref_cnt = ref_cnt[::-1]
np_cnt = np.concatenate((np_cnt, ref_cnt))

# bottom
ref_cnt = np.array(affinity.scale(MultiPoint(np_cnt), xfact=1, yfact=-1)) # reflecting full bottom part
ref_cnt[:,1] = ref_cnt[:,1]-(np.max(np_cnt[:,1])-np.min(np_cnt[:,1]))
ref_cnt = ref_cnt[::-1]
np_cnt = np.concatenate((np_cnt, ref_cnt))

bc_x, bc_y = np.min(np_cnt[:,0]), np.min(np_cnt[:,1])
sh_cnt = MultiPoint(np_cnt)
sh_cnt = affinity.translate(sh_cnt, xoff=-bc_x, yoff=-bc_y) # move to (0,0)
np_cnt = np.array(sh_cnt)

rect = patches.Rectangle((0, 0), real_x, real_y, linewidth=1, edgecolor='r', facecolor='none')
ax = plt.gca()
ax.add_patch(rect)
plt.plot(np_cnt[:,0], np_cnt[:,1], marker='.')
# plt.xticks(ticks=np.arange(0, real_x+1, .25), labels=[])
# plt.yticks(ticks=np.arange(0, real_y+1, .25), labels=[])
# plt.xlim([0, int(real_x)+1])
# plt.ylim([0, int(real_y)+1])
plt.grid()

In [ ]:
dx, dy = .25, .25
nx, ny = int(real_x/dx)+1, int(real_y/dy)+1
mask = np.empty([nx, ny])
poly_cnt = Polygon(sh_cnt)
for i in range(nx):
    for j in range(ny):
        p = Point(i*dx, j*dy)
        mask[i][j] = 1 if p.touches(poly_cnt) or p.within(poly_cnt) else 0
        
# mask = np.transpose(mask)

In [ ]:
# REFINEMENT

for i in np.arange(1, len(mask)-1):
    for j in np.arange(1, len(mask[i])-1):
        if mask[i][j] != 1:
            if (mask[i-1][j] == 1 and mask[i+1][j] == 1 and mask[i][j+1] == 1) or \
            (mask[i][j-1] == 1 and mask[i-1][j] == 1 and mask[i+1][j] == 1) or \
            (mask[i-1][j] == 1 and mask[i][j+1] == 1 and mask[i][j-1] == 1) or \
            (mask[i+1][j] == 1 and mask[i][j+1] == 1 and mask[i][j-1] == 1) or \
            (mask[i-1][j] == 1 and mask[i+1][j] == 1) or \
            (mask[i][j+1] == 1 and mask[i][j-1] == 1):
                print(mask[i-1:i+2,j-1:j+2])
                print("%d %d" % (i, j))
                mask[i][j] = 1
        if mask[i][j] == 1:
            if mask[i-1][j] == 0 and mask[i+1][j] == 0 and mask[i][j+1] == 0 and mask[i][j-1] == 0:
                mask[i][j] = 0

fig = figure(num=None, figsize=(10, 6), dpi=80, facecolor='w')
plt.imshow(mask, origin='lower', interpolation='none')

# mask = np.transpose(mask)
np.savetxt("/misc/home/u1220/lvd_aniso/mol_a_2nd/scars/scar.dat", mask, fmt="%d", comments="", header="%d %d" % (len(mask[0]), len(mask)))
# np.savetxt("/misc/home/u1220/analysis_jupyter/_trash/scar/scar.dat", mask, fmt="%d", comments="", header="%d %d" % (len(mask[0]), len(mask)))

In [ ]:
inmask = np.loadtxt("/misc/home/u1220/lvd_aniso/2D_update/test/id_36/debug_scar.dat", skiprows=1)
# inmask = np.loadtxt("/misc/home/u1220/lvd_aniso/2D_update/scar.dat", skiprows=1)

print("size %dx%d" % (len(inmask), len(inmask[0])))
fig = figure(num=None, figsize=(10, 6), dpi=80, facecolor='w')
plt.imshow(inmask, origin='lower')

In [ ]:
from shapely import affinity
from shapely.geometry import MultiPoint, Polygon, Point

rec_obs= np.array([[0,0], [1,0], [1,1], [0,1]])
len_x, len_y = (np.max(rec_obs[:,0])-np.min(rec_obs[:,0])), (np.max(rec_obs[:,1])-np.min(rec_obs[:,1]))
real_x = 20
real_y = real_x*(len_y/len_x)
scale_x, scale_y = real_x/len_x, real_y/len_y
rec_obs = np.array(affinity.scale(MultiPoint(rec_obs), xfact=scale_x, yfact=scale_y, origin=(0,0)))
fig = figure(num=None, figsize=(10, 6), dpi=80, facecolor='w')
plt.plot(rec_obs[:,0], rec_obs[:,1])
plt.grid()

In [ ]:
dx, dy = .25, .25
# real_x, real_y = 12, 12
nx, ny = int(real_x/dx), int(real_y/dy)
mask = np.empty([nx, ny])
poly_rec = Polygon(MultiPoint(rec_obs))
for i in range(nx):
    for j in range(ny):
        p = Point(i*dx, j*dy)
        mask[i][j] = 1 if p.touches(poly_rec) or p.within(poly_rec) else 0
        
# mask = np.transpose(mask)
fig = figure(num=None, figsize=(10, 6), dpi=80, facecolor='w')
plt.imshow(mask, origin='lower', vmin=0)

np.savetxt("/misc/home/u1220/lvd_aniso/mol_a_2nd/scars/obs_20mm.dat", mask, fmt="%d", comments="", header="%d %d" % (len(mask[0]), len(mask)))